import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [3]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

Load Data

In [4]:
train_dataset = datasets.MNIST(root='./datatset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,)
test_dataset = datasets.MNIST(root='./datatset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True,)

Create Fully Connected Network

In [5]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)

        return out

Save and Load function

Initialize Model

In [6]:
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device=device)

Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    losses = []
        
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data and target to the correct device (CPU or GPU)
        data = data.to(device)
        target = target.to(device)

        # Forward pass
        scores = model(data)
        loss = criterion(scores, target)
        losses.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training information
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{total_step}], Loss: {loss.item():.4f}")

Epoch [1/5], Step [1/938], Loss: 2.3024
Epoch [1/5], Step [2/938], Loss: 2.2867
Epoch [1/5], Step [3/938], Loss: 2.2963
Epoch [1/5], Step [4/938], Loss: 2.2609
Epoch [1/5], Step [5/938], Loss: 2.2675
Epoch [1/5], Step [6/938], Loss: 2.2553
Epoch [1/5], Step [7/938], Loss: 2.2538
Epoch [1/5], Step [8/938], Loss: 2.2649
Epoch [1/5], Step [9/938], Loss: 2.2489
Epoch [1/5], Step [10/938], Loss: 2.2509
Epoch [1/5], Step [11/938], Loss: 2.2186
Epoch [1/5], Step [12/938], Loss: 2.1927
Epoch [1/5], Step [13/938], Loss: 2.2202
Epoch [1/5], Step [14/938], Loss: 2.1502
Epoch [1/5], Step [15/938], Loss: 2.1667
Epoch [1/5], Step [16/938], Loss: 2.1130
Epoch [1/5], Step [17/938], Loss: 2.0934
Epoch [1/5], Step [18/938], Loss: 2.0759
Epoch [1/5], Step [19/938], Loss: 2.0929
Epoch [1/5], Step [20/938], Loss: 2.1036
Epoch [1/5], Step [21/938], Loss: 2.0294
Epoch [1/5], Step [22/938], Loss: 2.0504
Epoch [1/5], Step [23/938], Loss: 2.0266
Epoch [1/5], Step [24/938], Loss: 1.9834
Epoch [1/5], Step [25/938

In [9]:
def evaluate(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples=0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _,pred = scores.max(1)
            num_correct += (pred == y).sum()
            num_samples += pred.size(0)

        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
    model.train()

In [10]:
evaluate(train_loader, model)
evaluate(test_loader, model)

Checking accuracy on training data
Got 59028/60000 with accuracy 98.38
Checking accuracy on test data
Got 9844/10000 with accuracy 98.44
